<a href="https://colab.research.google.com/github/kfir1g/AI-Project/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

#Initialize tensorflow-GPU

In [0]:
!pip install tensorflow-gpu
import tensorflow as tf 

##This part makes sure that we are working on GPU in runtime

In [0]:

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at : {}'.format(device_name))  

Found GPU at : /device:GPU:0


#Data Preprocessing

##importing Libraries and Data

##Import the data
data is the results of each tree on each line of the testing data + the actual value

In [0]:
import numpy as np
import pandas as pd
dataset = pd.read_csv("/content/gdrive/My Drive/AIProj/testing_by_tree_1_all.csv") #Change to the path of data 
#dataset = pd.read_csv("/content/gdrive/My Drive/AIProj/All_trees_with_random_forst.csv") #choose differnt file - choose the data which combined with random forest
dataset = dataset.fillna(value=-32)#Min value for Nan values in the trees

#Getting new array for prediction

In [0]:
import numpy as np
import pandas as pd

Sepdataset = pd.read_csv("/content/gdrive/My Drive/AIProj/All_trees_with_random_forest_September.csv")
Sepdataset = Sepdataset.fillna(value = -32) 

In [0]:
X = Sepdataset.iloc[:,0:-1].values 

In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

##Getting the dependent and independent variables

splits the information to results from tree and actual result

In [0]:
X = dataset.iloc[:,1:-1].values 
y = dataset.iloc[:,-1].values
myMin = -25 #Getting this min from the data, could be changed for differnt data 
y = y + myMin*-1 +1

In [0]:
MAX_PICKUP_BIKES = y.max() + 1 
NUM_OF_INPUTS = X.shape[1]
MAX_PICKUP_BIKES = 58 #Choose from the data, could be changed
SHIFT = 26  # the min we decided + 1 

##Splitting the dataset into training set and Test set 

Splits the information to training and test for the NN, test_size can be changed

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

##Random forest file split

In [0]:
X_train = X_train[:,0:-1]
test_random_forest = X_test[:,-1]
X_test = X_test[:,0:-1]

##Feature scaling - not needed for trees

In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_Y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


#Builed the NN

##Parameters for NN
softmax - output neuron that got the highest value
Epoch - how many time will go over all the data
Batch_size - update weights every batch inputs
Can add more layers or change the numbers

In [0]:
FIRST_LAYAR = 100
SECONED_LAYAR = 60
OUTPUT_ACTIVATION_FUNC = 'softmax'
OPTIMIZER = 'adam'            
NUM_EPOCH = 100
BATCH_SIZE = 60

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
classifier = Sequential()

classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform', activation = 'tanh', input_dim = NUM_OF_INPUTS))
classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'tanh' ))
classifier.add(Dense(output_dim = MAX_PICKUP_BIKES, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
 

#Define Optimizer and Loss function 

##Tuning the optimiers, and compiling the classifier

In [0]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay= 0, momentum=0.9, nesterov=True) #1e-6
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
rms = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
classifier.compile(optimizer = OPTIMIZER , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy']) 

#Train the data 

##fit is the training, after that saves the weights

In [0]:
classifier.fit(X_train, Y_train, batch_size = BATCH_SIZE , nb_epoch = NUM_EPOCH)
from keras.models import load_model
classifier.save_weights('/content/gdrive/My Drive/AIProj/my_model_weights.h5')

#Load Wights 
To Load wights you need to create the model again and use the classifier to predict  - need to be tested

In [0]:

MAX_PICKUP_BIKES = 58 # y.max() +1 #Todo: check what is the max number
NUM_OF_INPUTS = X_train.shape[1]
FIRST_LAYAR = 100
SECONED_LAYAR = 60
OUTPUT_ACTIVATION_FUNC = 'softmax'
OPTIMIZER = 'adam' #'adam' #stochastich gradient descent
NUM_EPOCH = 100
BATCH_SIZE = 60

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
classifier = Sequential()

classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform',activation ='tanh',input_dim = NUM_OF_INPUTS ))
classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'tanh' ))
classifier.add(Dense(output_dim = MAX_PICKUP_BIKES, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
classifier.load_weights('/content/gdrive/My Drive/AIProj/my_model_weights.h5') # Change to the path of the wights

#How to predict one input: 

#input_vector = #put input vector you want to test
#Y_pred = classifier.predict(input_vector)
#pickup_predection = Y_pred[0].argmax()


#Extract the predctions 


1.   getting the predection from the classifier
2.   extracting the argmax which is the number of picups fo reach input
3.  checking while using the test vector our results




##Get predction for new matrix

In [0]:
Y_pred = classifier.predict(X)
Y_get = []
for i in range(len(Y_pred)):
  Y_get.append(Y_pred[i].argmax() - SHIFT)

##adding the vector predction to the file

In [0]:
Sepdataset["demand"] = Y_get
Sepdataset.to_csv("/content/gdrive/My Drive/AIProj/Final.csv")


##Extract results for our training 

In [0]:
Y_pred = classifier.predict(X_test)
Y_get = []
for i in range(len(Y_pred)):
  Y_get.append(Y_pred[i].argmax())
a = np.asarray(Y_get)
np.savetxt("/content/gdrive/My Drive/AIProj/Pred.csv",a,delimiter = ",")


In [0]:
total = 0 
correct = 0 
wrong = 0 
histo_pred = []
for i in range(len(Y_pred)):
  total = total +1
  if(Y_get[i] == Y_test[i]):
     correct = correct + 1
  else:
    histo_pred.append(abs(Y_get[i] - Y_test[i] ))
    wrong = wrong + 1

    
print("Total "+ str(total) )
print("Correct " + str(correct))
print("Wrong " + str(wrong))
print("acc:" + str(float(correct)/total) )
print(max(histo_pred))    

##Histogram for out test prediction

In [0]:
hist = pd.Series(histo_pred).value_counts()
plt.bar(x=hist.index, height=hist)

##Extract results for random forest

In [0]:
total = 0 
correct = 0 
wrong = 0 
histo_random_forest = []
for i in range(len(test_random_forest)):
  total = total +1
  if(test_random_forest[i] == Y_test[i]):
     correct = correct + 1
  else:
    histo_random_forest.append(abs(test_random_forest[i] - Y_test[i] ))
    wrong = wrong + 1

    
print("Total "+ str(total) )
print("Correct " + str(correct))
print("Wrong " + str(wrong))
print("acc:" + str(float(correct)/total) )
print(max(histo_random_forest))    

##Histogram for random_forest

In [0]:
hist = pd.Series(histo_random_forest).value_counts()
plt.bar(x=hist.index, height=hist)

#Evaluating the ANN 

##Using K-Fold Cross Validation 
The main idea is dividing to 10 folds and taking the mean and variance, the mean is the actual result of our ANN and the variance will indicate if we are overfitting
NN needs to be the same as before

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
  classifier = Sequential()

  classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform',activation ='tanh',input_dim = NUM_OF_INPUTS ))
  classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'tanh' ))
  classifier.add(Dense(output_dim = MAX_PICKUP_BIKES, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
  classifier.compile(optimizer = 'adam'  , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = BATCH_SIZE, nb_epoch = NUM_EPOCH  )
accuracies = cross_val_score(estimator = classifier, X = X_train, y = Y_train, cv =10, n_jobs = -1)# cv num of folds , n_jobs =-1 using all the GPU
#Accuracies is a vector with all the #folds results(in our case 10) 
#The real Result is the mean of this vector, also we need to check for hight or low variance to know if our results are good
mean = accuracies.mean()
variance = accuracies.std()
variance, mean
  

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


(0.002268920817533607, 0.6896752746592789)

##Tuning The ANN - choose the best param from all the options


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer,first, seconed ):
  classifier = Sequential()

  classifier.add(Dense(output_dim = first  , init = 'uniform',activation ='tanh',input_dim = NUM_OF_INPUTS ))
  classifier.add(Dense(output_dim = seconed, init = 'uniform', activation = 'tanh' ))
  classifier.add(Dense(output_dim = MAX_PICKUP_BIKES, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
  classifier.compile(optimizer = optimizer  , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 50,75, 150 ],'epochs': [25], 'optimizer' : ['adam', 'rmsprop','sgd'], 'first': [100,200], 'seconed':[100,200]}
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)  
grid_serach = grid_search.fit(X_train, Y_train)                     
best_parameters = grid_serach.best_params_
best_accuracy = grid_search.best_score_
best_parameters, best_accuracy